# Transfer Learning - Feature Extraction

<font color='steelblue'>

<font size = 4>
    
Use *`state of the art`* image classification models from Tensorflow Hub<br><br>
</font>

</font>

<font size = 3>
Here we are going to leverage image classification weights that are learnt by these predefined models, <br>
    
**Following is included here:<br>**
    
- `Load & Prepare` training and test images<br>
- `Identify` location of the predefined model
- `Create` model that will be applied to our data
- `Train & Evaluate` model
- `Explore` model performance
</font>

## Download the dataset

In [ ]:
import os
import shutil

In [ ]:
# define location of data
dpath = "../datasets/FoodClasses/"

In [ ]:
# list the directories in dataset
for dirpath, dirnames, filenames in os.walk(dpath):
    print(f"{len(dirnames)} directores and {len(filenames)} images in '{dirpath}'")

<font size = 4>
    
**Note:**<br>
- Training directories have `75 images` each where as the test directories have `250 images` each
- Training on `less data`, but evaluation has `more data`

</font>

## Prepare Data<br>

<font size = 4>

**Following steps in preparing data:** <br>
    
- Use `ImageGenerator` class to normalize the data
- Use `flow_from_directory` to load images and their classes

[`ImageGenerator`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image)
    
</font>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

In [ ]:
trainDir = dpath + "train"
testDir = dpath + "test"
trainDir, testDir

In [ ]:
# normalize images
trainGen = ImageDataGenerator(rescale = 1/255.0)

In [ ]:
# normalize images
testGen = ImageDataGenerator(rescale = 1/255.0)

## Load Data<br>
<font size = 4>


Parameters to use from the `flow_from_directory()`:
- `directory`    - the file path of the target directory for images
- `target_size`  - the target image size that we want in our dataset
- `batch_size`   - how many images we want to load at a time, e.g. `default is 32`, load 32 images and their labels
- `class_mode`   - One hot encoded labels `categorical` is default

[`tf.keras.preprocessing` Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
    
</font>

In [ ]:
print("Training Data")
trainData = trainGen.flow_from_directory(trainDir,
                                         target_size = IMAGE_SHAPE,
                                         batch_size = BATCH_SIZE,
                                         class_mode = "categorical")

In [ ]:
print("Test Data")
testData = testGen.flow_from_directory(testDir,
                                      target_size = IMAGE_SHAPE,
                                      batch_size = BATCH_SIZE,
                                      class_mode = "categorical")

In [ ]:
trainData.classes.shape

In [ ]:
trainData.class_indices

In [ ]:
trainData

In [ ]:
# example of batch data (taking one batch - size is 32)
images, labels = trainData[0]
print(labels, images)

Label above is `one hot encoded` `[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]`is `grilled_salmon`

## Create Models using Tensorflow Hub<br>

<font size = 3>
    
- Until now we have built our models from scratch where we have defined our neural network layers
- Follow similar process of creating layers, except the layers are going to come from [Tensorflow Hub](https://tfhub.dev)

- Use model from tensorflow hub call [EfficientNet](https://arxiv.org/abs/1905.11946)
    
Typically these are the steps to follow when selecting a model:
1. Go to [Tensorflow Hub](https://tfhub.dev)
2. Select problem domain (here it is image processing)
3. Doing classification here
4. Check out the different models that are available
5. Select a particular model then checkout different versions
6. Check out which version suites the data set image size (here it is `224`)
    
    
[EfficientNet](https://tfhub.dev/google/collections/efficientnet/1)

[EfficientNet v1](https://tfhub.dev/google/collections/efficientnet_v2/1)
    
    
</font>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# version 1 of the model

effNetURL = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# version 2 of the model
#effNetURL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

## Feature Extraction Transfer Learning<br>

<font size = 3>

- Here use the underlying `patterns (weights)` that have been learnt by the `pre-defined model` and apply it to `our output classes`
- `EfficientNetB0 has 236 layers` and has been `trained on 1000 classes`
- To use it for our data, `remove certain layer(s)` and `replace it with a layer` that has the `right number of output classes` in our dataset
- `Few layers become trainable`, the `rest of the layers are frozen`
- `Technique is useful when our data is similar` to the data that the model was trained on
    
</font>

## Using predefined model<br>

<font size = 3>

- The URL above are saved pretained models on tensorflow hub
- When we use these models, they are automatically downloaded
- Need `KerasLayer()` from the tensorflow hub library
- Write a function that creates a model using the keras layer

    
</font>

In [ ]:
# Function to create a model
def createModel(modelUrl, numClasses = 10):
    """
    Takes a tensorflow hub url and create a Keras Sequential model
    
    Args:
      modelUrl (string): URL for feature extraction
      numClasses (int): number of neurons in the output layer i.e. number of classes
      
    Returns:
      An uncompiled Keras Sequential model
    """
    
    # download the predefined model  from URL and save as Keras Layer
    # Shape is defined as 2D, but images are colored, hence we have
    # to add dimension of 3, and the rest is for batch
    featureExtractLayer = hub.KerasLayer(modelUrl,
                                        trainable = False, # freeze the layers
                                        name = 'feature_extraction_layer', 
                                        input_shape = IMAGE_SHAPE + (3,))
    
    model = tf.keras.Sequential([
        featureExtractLayer,                # use as base
        layers.Dense(numClasses, activation = 'softmax', name = "output_layer")
    ],name = "FeatureExtraction")
        
        
    
    return model

In [ ]:
import matplotlib.pyplot as plt

# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

## Model creation and Training

In [ ]:
# create model
tf.random.set_seed(2345)

effNetModel = createModel(effNetURL, numClasses = len(trainData.class_indices))

In [ ]:
# compile model
effNetModel.compile(loss = "categorical_crossentropy",
                   optimizer = tf.keras.optimizers.Adam(),
                   metrics = ["accuracy"])

In [ ]:
effNetModel.summary()

In [ ]:
(len(trainData), len(testData))

In [ ]:
%%time
# train model (could take upto 24+ minutes)
effNetHistory = effNetModel.fit(trainData,
                               epochs = 5,
                               steps_per_epoch = len(trainData),
                               validation_data = testData,
                               validation_steps = len(testData),
                               verbose = 1)

## Model Peformance

In [ ]:
plot_loss_curves(effNetHistory)

In [ ]:
effNetModel.summary()

In [ ]:
effNetModel.weights

<font color = 'steelblue'>
<font size = 4>

How cool is it! <br><br>
- Both accuracy and validation accuracy *`85% or more`*, just using predefined model
- Also, there was more validation data then training data !
- With couple of lines of code, leverage *`state of the art`* model
    
</font>
</font>